# 0. Load libraries/dataset, clean dataset, and create some useful functions.

#### Load libraries/dataset

In [1]:
# Load libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from os.path import isfile
from itertools import product

from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error as sk_mae, mean_squared_error as sk_mse
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

sns.set(style='ticks', color_codes=True)

# Load dataset
data_path = './datasets/Suicide_Rates.csv'

suicides = pd.DataFrame()
if not isfile(data_path):
    print("Dataset not found. Please check that the dataset exists and the path is correct.")
    
else:
    suicides = pd.read_csv(data_path)
    
display(suicides)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z


#### Clean dataset

In [2]:
# Adjust dataset to taste    
# GDP per year has a weird clerical error and is recorded as strings instead of ints
suicides = suicides.rename(columns={' gdp_for_year ($) ': 'gdp_for_year ($)'})

suicides['gdp_for_year ($)'] = suicides['gdp_for_year ($)'].str.replace(',', '')
suicides['gdp_for_year ($)'] = suicides['gdp_for_year ($)'].apply(int)

# Lowercase HDI for uniformity
suicides = suicides.rename(columns={'HDI for year': 'hdi for year'})

# Define the target feat
target_feat = 'suicides/100k pop'

In [3]:
# Impute HDI
hdi_mean = suicides['hdi for year'].mean()
suicides['hdi for year'] = suicides['hdi for year'].fillna(hdi_mean)

# Drop extraneous features
extra_feats = ['country', 'suicides_no', 'population', 'country-year',
               'gdp_for_year ($)', 'gdp_per_capita ($)', 'year', 'hdi for year']
suicides.drop(columns=extra_feats, inplace=True)

# Reorganize columns alphabetically
reorg = lambda df: df.reindex(sorted(df.columns), axis=1)
suicides = reorg(suicides)

display(suicides)

,age,generation,sex,suicides/100k pop
0,15-24 years,Generation X,male,6.71
1,35-54 years,Silent,male,5.19
2,15-24 years,Generation X,female,4.83
3,75+ years,G.I. Generation,male,4.59
4,25-34 years,Boomers,male,3.28
...,...,...,...,...
27815,35-54 years,Generation X,female,2.96
27816,75+ years,Silent,female,2.58
27817,5-14 years,Generation Z,male,2.17
27818,5-14 years,Generation Z,female,1.67


In [4]:
# Create test df
test_df = pd.DataFrame(data=[['15-24 years', 'male', 'Generation X']], columns=['age', 'sex', 'generation'])
test_df = reorg(test_df)

display(test_df)

,age,generation,sex
0,15-24 years,Generation X,male


#### Create some useful functions (also define model)

In [5]:
# Mean Absolute Error (from module07 notebook)
def mae(_y, _y_pred):
    return (len(_y)**-1) * np.sum(np.abs(_y_pred-_y))

# Cross validation alias
sk_mae = make_scorer(sk_mae)
sk_mse = make_scorer(sk_mse)
scorers = {'sk_mae': sk_mae, 'sk_mse': sk_mse}
cross_val = lambda _model, x, y: cross_validate(_model, x, y, n_jobs=4, scoring=scorers)

# Split data
def split_XYdata(df, target=target_feat):
    X = df.loc[:, df.columns != target].values
    Y = df.loc[:, df.columns == target].values
    
    return X, Y
    
# Split, fit, and predict (return split and prediction)
def split_fit_predict(model, data_df, test_df):
    X, Y  = split_XYdata(data_df)

    model.fit(X, Y)

    y_pred = model.predict(test_df)
    
    return X, Y, y_pred

# Show the results
def display_results(model, X, Y, y_pred):
    # Show the predicted suicide rate
    print(f"Predicted Suicide Rate: {y_pred[0][0]}\n")
    
    # Check MAE
    print(f"MAE: {mae(Y, y_pred)}\n")
    
    # Perform Cross Validation
    this_cv = cross_val(model, X, Y)
    print(f"Cross-validation MAE scores:\n{this_cv['test_sk_mae'].tolist()}\n")
    print(f"Cross-validation MSE scores:\n{this_cv['test_sk_mse'].tolist()}\n")
    
    # Check number of and values of coefficients
    print(f"There are {len(model.coef_[0])} coefficients:")
    print(model.coef_[0])

In [6]:
# Function for encoding dataset and testset into one-hots

# I did this the long way around, but now we can build one-hot encoded
# test datapoints form readable dataframes.
def onehot_encode(data_df, test_df, extra_classes={}):
    for feat in data_df.loc[:, data_df.columns != target_feat]:
        lb = LabelBinarizer()

        # fit the binarizer
        lb = lb.fit(data_df[feat])
        
        # append any specified classes
        if feat in extra_classes:
                classes = lb.classes_.tolist()
                classes += extra_classes[feat]
                
                lb.classes_ = np.array(classes)

        # get the column names
        cols = lb.classes_
        cols = [feat + ' - ' + col for col in cols]

        # get the transforms
        data_trans = lb.transform(data_df[feat])
        test_trans = lb.transform(test_df[feat])

        # LabelBinarizer doesn't split 'sex' into an array :(
        # I ran a model with pd.get_dummies and this doesn't seem to effect it too bad (+/- 0.2 on the prediction)
        if feat == 'sex':
            data_df[feat] = data_trans
            test_df[feat] = test_trans
            continue

        # tranform the feat column
        data_trans = pd.DataFrame(data_trans, columns=cols)
        data_df = pd.concat([data_df, data_trans], axis=1)
        data_df.drop(columns=[feat], inplace=True)

        # do the same to the test
        test_trans = pd.DataFrame(test_trans, columns=cols)
        test_df = pd.concat([test_df, test_trans], axis=1)
        test_df.drop(columns=[feat], inplace=True)
        
    return data_df, test_df

In [7]:
# Function for running the LabelEncoder on dataset and testset
def label_encode(data_df, test_df, extra_classes={}):
    for feat in data_df.loc[:, data_df.columns != target_feat]:
        le = LabelEncoder()

        le = le.fit(data_df[feat])
        
        # append any specified classes
        if feat in extra_classes:
            classes = le.classes_.tolist() 
            classes += extra_classes[feat]
                               
            le.classes_ = np.array(classes)

        data_df[feat] = le.transform(data_df[feat])
        test_df[feat] = le.transform(test_df[feat])
        
    return data_df, test_df

In [8]:
# Define our model(s)
model = LinearRegression(n_jobs=4, normalize=True, copy_X=True)

# 1. Perform multiple linear regression with one-hot encoded variables.

__*Summary of Results*__

The number of linear coefficients is 13 (the number of the independent variables).

The predicted suicide rate for 20-year old, Generation-X males is around 16.89 / 100k pop.

The MAE for this prediction is 14.38.

In [9]:
# Copy the dataframes
suic_oh = suicides.copy()
test_oh = test_df.copy()

# Run the encoder
suic_oh, test_oh = onehot_encode(suic_oh, test_oh)

# Alphabetize the columns in the dataframes
suic_oh = reorg(suic_oh)
test_oh = reorg(test_oh)
                
display(suic_oh, test_oh)

,age - 15-24 years,age - 25-34 years,age - 35-54 years,age - 5-14 years,age - 55-74 years,age - 75+ years,generation - Boomers,generation - G.I. Generation,generation - Generation X,generation - Generation Z,generation - Millenials,generation - Silent,sex,suicides/100k pop
0,1,0,0,0,0,0,0,0,1,0,0,0,1,6.71
1,0,0,1,0,0,0,0,0,0,0,0,1,1,5.19
2,1,0,0,0,0,0,0,0,1,0,0,0,0,4.83
3,0,0,0,0,0,1,0,1,0,0,0,0,1,4.59
4,0,1,0,0,0,0,1,0,0,0,0,0,1,3.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27815,0,0,1,0,0,0,0,0,1,0,0,0,0,2.96
27816,0,0,0,0,0,1,0,0,0,0,0,1,0,2.58
27817,0,0,0,1,0,0,0,0,0,1,0,0,1,2.17
27818,0,0,0,1,0,0,0,0,0,1,0,0,0,1.67


,age - 15-24 years,age - 25-34 years,age - 35-54 years,age - 5-14 years,age - 55-74 years,age - 75+ years,generation - Boomers,generation - G.I. Generation,generation - Generation X,generation - Generation Z,generation - Millenials,generation - Silent,sex
0,1,0,0,0,0,0,0,0,1,0,0,0,1


### Results

In [10]:
# Split the data, fit with all, predict on test, and show results
X, Y, y_pred = split_fit_predict(model, suic_oh, test_oh)

display_results(model, X, Y, y_pred)

Predicted Suicide Rate: 16.89453125

MAE: 14.382717480679366

Cross-validation MAE scores:
[10.453020639097323, 9.913490631739757, 11.738330759121135, 9.81587890625, 9.319327849793314]

Cross-validation MSE scores:
[234.36399814484167, 210.427496811885, 360.18902712541876, 250.3852204067151, 227.16417978285483]

There are 13 coefficients:
[-1.16779853e+13 -1.16779853e+13 -1.16779853e+13 -1.16779853e+13
 -1.16779853e+13 -1.16779853e+13  4.44580855e+13  4.44580855e+13
  4.44580855e+13  4.44580855e+13  4.44580855e+13  4.44580855e+13
  1.48332376e+01]


# 2. Perform Multiple Linear Regression with numerical variables.

__*Summary of Results*__

The number of linear coefficients is 3 (the number of the independent variables).

The predicted suicide rate for 20-year old, Generation-X males is around 15.18 / 100k pop.

The MAE for this prediction is 13.55.

In [11]:
# Make a new dataframe
suic_num = suicides.copy()
test_num = test_df.copy()

# Encode the data
suic_num, test_num = label_encode(suic_num, test_num)

# Alphabetize the columns in the dataframes
suic_num = reorg(suic_num)
test_num = reorg(test_num)

display(suic_num, test_num)

,age,generation,sex,suicides/100k pop
0,0,2,1,6.71
1,2,5,1,5.19
2,0,2,0,4.83
3,5,1,1,4.59
4,1,0,1,3.28
...,...,...,...,...
27815,2,2,0,2.96
27816,5,5,0,2.58
27817,3,3,1,2.17
27818,3,3,0,1.67


,age,generation,sex
0,0,2,1


### Results

In [12]:
# Split the data, fit with all, predict on test, and show results
X, Y, y_pred = split_fit_predict(model, suic_num, test_num)

display_results(model, X, Y, y_pred)

Predicted Suicide Rate: 15.178500202776634

MAE: 13.554067562029791

Cross-validation MAE scores:
[10.854117034956143, 10.56279668035128, 12.7309240899968, 10.308585234299116, 10.323692734594225]

Cross-validation MSE scores:
[256.5655901237479, 243.5716118277855, 402.42963947153413, 281.90917219317447, 271.32204060998185]

There are 3 coefficients:
[ 2.30253066 -0.98903628 14.84646226]


# 3. Are there any changes between the two models?

There is very little change between the models. There are obviously fewer coefficients in the numerically encoded dataset, and numerical encoding does seems to produce closer results based on MAE scores.

# 4. What is the prediction for age 33, male and generation Alpha?

Linear regression will extrapolaite on new continuous data but not on new categorical data. This is because the coefficient for the new category cannot be tuned as it is not present in the dataset, but the new continuous data will use the same coefficient. This is demonstrated in cells 14 and 15 below. 

__Summary of Results__

The one-hot produces a large negative number that is likely defined by sk-learn or the OS.

The numerically encoded data produces continuous data for generation and thus the Linear Regressor is able to extrapolaite. From this run we get a __suicide rate of 13.53__.

### Getting Ready

In [13]:
# define the extra class
extra = {'generation': ['Alpha']}

# Create new test df
test_df2 = pd.DataFrame(data=[['25-34 years', 'male', 'Alpha']], columns=['age', 'sex', 'generation'])
test_df2 = reorg(test_df2)

display(test_df2)

,age,generation,sex
0,25-34 years,Alpha,male


### One-hot Encoded Linear Regression

In [14]:
# Create new onehot encoded dataset
suic_oh2 = suicides.copy()
test_oh2 = test_df2.copy()

suic_oh2, test_oh2 = onehot_encode(suic_oh2, test_oh2, extra_classes=extra)

# Alphabetize columns
suic_oh2 = reorg(suic_oh2)
test_oh2 = reorg(test_oh2)

# Show the dataframes
display(suic_oh2, test_oh2)

# Split the data, fit with all, predict on test, and show results
X, Y, y_pred = split_fit_predict(model, suic_oh2, test_oh2)

display_results(model, X, Y, y_pred)

,age - 15-24 years,age - 25-34 years,age - 35-54 years,age - 5-14 years,age - 55-74 years,age - 75+ years,generation - Alpha,generation - Boomers,generation - G.I. Generation,generation - Generation X,generation - Generation Z,generation - Millenials,generation - Silent,sex,suicides/100k pop
0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,6.71
1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,5.19
2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,4.83
3,0,0,0,0,0,1,0,0,1,0,0,0,0,1,4.59
4,0,1,0,0,0,0,0,1,0,0,0,0,0,1,3.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27815,0,0,1,0,0,0,0,0,0,1,0,0,0,0,2.96
27816,0,0,0,0,0,1,0,0,0,0,0,0,1,0,2.58
27817,0,0,0,1,0,0,0,0,0,0,1,0,0,1,2.17
27818,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1.67


,age - 15-24 years,age - 25-34 years,age - 35-54 years,age - 5-14 years,age - 55-74 years,age - 75+ years,generation - Alpha,generation - Boomers,generation - G.I. Generation,generation - Generation X,generation - Generation Z,generation - Millenials,generation - Silent,sex
0,0,1,0,0,0,0,1,0,0,0,0,0,0,1


Predicted Suicide Rate: -11709671999268.717

MAE: 11709671999281.535

Cross-validation MAE scores:
[10.452455040213875, 9.935305647915168, 11.738655292393064, 9.828309219985622, 9.33589234363767]

Cross-validation MSE scores:
[234.38984912172123, 210.48999367719662, 359.9687754786159, 250.40787846037307, 227.0106721936202]

There are 14 coefficients:
[7.14047622e+12 7.14047622e+12 7.14047622e+12 7.14047622e+12
 7.14047622e+12 7.14047622e+12 9.29002979e+12 2.09997018e+13
 2.09997018e+13 2.09997018e+13 2.09997018e+13 2.09997018e+13
 2.09997018e+13 1.48508782e+01]


### Numerically Encoded Linear Regression

In [15]:
# Create new numerical dataset
suic_num2 = suicides.copy()
test_num2 = test_df2.copy()

suic_num2, test_num2 = label_encode(suic_num2, test_num2, extra_classes=extra)

# Alphabetize columns
suic_num2 = reorg(suic_num2)
test_num2 = reorg(test_num2)

# Show the dataframes
display(suic_num2, test_num2)

# Split the data, fit with all, predict on test, and show results
X, Y, y_pred = split_fit_predict(model, suic_num2, test_num2)

display_results(model, X, Y, y_pred)

,age,generation,sex,suicides/100k pop
0,0,2,1,6.71
1,2,5,1,5.19
2,0,2,0,4.83
3,5,1,1,4.59
4,1,0,1,3.28
...,...,...,...,...
27815,2,2,0,2.96
27816,5,5,0,2.58
27817,3,3,1,2.17
27818,3,3,0,1.67


,age,generation,sex
0,1,6,1


Predicted Suicide Rate: 13.524885751088624

MAE: 12.847395212349225

Cross-validation MAE scores:
[10.854117034956143, 10.56279668035128, 12.7309240899968, 10.308585234299116, 10.323692734594225]

Cross-validation MSE scores:
[256.5655901237479, 243.5716118277855, 402.42963947153413, 281.90917219317447, 271.32204060998185]

There are 3 coefficients:
[ 2.30253066 -0.98903628 14.84646226]


# 5. Give one advantage when using regression in terms of input data features.

One advantage is that we do not have to engineer data into bins (like high vs. low suicide rates). Besides reducing the engineering work, I think this also gives more interpretable results, as we don't have to define what a high rate is (we just have to know something like 30/100k pop is high).

# 6. Give one advantage when using regular numerical values rather than one-hot encoding for regression.

One advantage of numerical encoding is that manually inputing a datapoint to predict off of is easier and more readable. 

In cell 6, I defined a complicated method for one-hot encoding the dataset and testset, such that I could avoid having to do the nitty-gritty work of creating a testable array of binary values. The method I defined allows us to create a test dataframe with just the values we want to test with in an easy human-readable format. 

In cell 7, I did the same thing for label encoding, and it was both much easier to do and much shorter code-wise.

# 7. Would you recommend the customer a classifier or regressor for this problem?

I think I would recommend a regressor for this problem, as I think receiving a predicted suicide rate is preferable to receiving a predicted category (even if those technically encode the same data).

Additionally, I think we would want to frame our dataset for continuous data (Label Encode), as this will allow us to extrapolaite to new generations. We might also be able to get more accurate predictions on continuous data by fitting on GDP and HDI alongside the generational, age, and sex data.